#### Section - B

- In this project you will apply the AdaBoost boosting algorithm to implement an ensemble learning approach for solving a (binary) classification problem. 
- The (one- dimensional) training data set D is given in Table 4.12 on page 352 of the textbook. The base classifier is a simple, one-level decision tree (decision stump) (as explained on p. 303 of the textbook).
- Determine the number of boosting rounds and show the result of each round (the probability distribution pi’s at each round, the records chosen at each round, the model (tree) obtained at each round, the ε and the α at each round), as well as the result obtained on D with the final ensemble classifier. 
- Note that the textbook uses the notation w (weight) for what we called p (probability) in the derivation we did in the lectures. (The textbook has quite a few typos!) Also, do not forget the stopping condition we discussed.
- What is the result of running your ensemble classifier on the following test data? X = 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0


Submit source code and your output (the round-wise results and the result on the test data) following the styles of Figures 4.46, 4.49, and 4.50 of the textbook.

In [1]:
import pandas as pd

#### Section - A : 1. Load datasets

In [24]:
training_dataset_df = pd.read_csv("dataset_training_table_3.36.csv")
training_dataset_df

Instance  A  B  C Class
0         1  0  0  0     +
1         2  0  0  1     +
2         3  0  1  0     +
3         4  0  1  1     -
4         5  1  0  0     +
5         6  1  0  0     +
6         7  1  1  0     -
7         8  1  0  1     +
8         9  1  1  0     -
9        10  1  1  0     -

In [25]:
validation_dataset_df = pd.read_csv("dataset_validation_table_3.36.csv")
validation_dataset_df

Instance   A   B   C Class
0         11   0   0  0     +
1         12   0   1  1     +
2         13   1   1  0     +
3         14   1   0  1     -
4         15   1   0  0     +

##### Replace class labels "+" and "-" with 1 and 0

In [28]:
training_dataset_df['Class'] = training_dataset_df['Class'].map({'+': 1, "-": 0})
training_dataset_df

Instance  A  B  C  Class
0         1  0  0  0      1
1         2  0  0  1      1
2         3  0  1  0      1
3         4  0  1  1      0
4         5  1  0  0      1
5         6  1  0  0      1
6         7  1  1  0      0
7         8  1  0  1      1
8         9  1  1  0      0
9        10  1  1  0      0

In [14]:
validation_dataset_df['Class'] = validation_dataset_df['Class'].map({'+': 1, "-": 0})
validation_dataset_df

Instance   A   B   C  Class
0         11   0   0  0      1
1         12   0   1  1      1
2         13   1   1  0      1
3         14   1   0  1      0
4         15   1   0  0      1

#### Section A - 2. Task

- Determine the number of base classifiers (or bagging rounds) and 
- show the result of each round (the records chosen at each round and the model (tree) obtained at each 
round). 
- What is the result of running your ensemble classifier on the data set given in the 
second table (labeled “Validation”) in Fig 3.36 on page 189 of the textbook? 

Bagging (Bootstrap Aggregation) is used when our goal is to reduce the variance of a decision tree. Here idea is to create several subsets of data from training sample chosen randomly with replacement. Now, each collection of subset data is used to train their decision trees. As a result, we end up with an ensemble of different models. Average of all the predictions from different trees are used which is more robust than a single decision tree

**Algorithm from textbook:**
    
1.Let k be the number of bootstrap samples.

2: fori=1 to k do

    3: Create a bootstrap sample of size N, Dt.
    
    4: Train a base classifierCa on the bootstrap sample Di.
    
5: end for

6: C. (r) : argmax1,6(Ct1r1 : y) . c

{d(') : 1 if its argument is true and 0 otherwise}.

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
from collections import Counter

class BaggingClassifier:
    
    def train(self, XTRAIN, YTRAIN, k_base_estimators):
        
        self.models = list()
        
        for ith_estimator in k_base_estimators:
            this_base_classifier = DecisionTreeClassifier()
            bootstrap_sample = np.random.choice(np.arange(len(XTRAIN)),
                                               size=len(XTRAIN),
                                               replace=True)
            bootstrap_XTRAIN = XTRAIN[bootstrap_sample]
            bootstrap_YTRAIN = YTRAIN[bootstrap_sample]
            this_base_classifier.fit(XTRAIN, YTRAIN)
            self.models.append(this_base_classifier)
        
        
    def predict(self, XTEST):
        y_predictions = list()
        for test_record in XTEST:
            k_classifier_votes = []
            for this_trained_model in self.models:
                k_classifier_votes.append(this_trained_model.predict(test_record))
                
            k_classifier_votes_counter = Counter(lst)
            prediction = k_classifier_votes_counter.most_common(1)[0][0]
            y_predictions.append(prediction)
        return y_predictions

In [32]:
ensemble_classifier = BaggingClassifier()

##### Split Train and Test Set